<H1>Предсказание Стоимости Недвижимости. Предсказание Типа Недвижимости (Новостройка, Вторичка).</H1>

<H2>Библиотеки.</H2>

In [18]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.preprocessing
import sklearn.metrics
import sklearn.model_selection
import sklearn.ensemble
from tensorflow import keras
import joblib

<H2>Файл Property.csv</H2>

In [19]:
FilePath = 'Property.csv'
Data = pd.read_csv(FilePath)
Data = Data.dropna()
Data = Data.drop_duplicates()
display(Data.head())

,Unnamed: 0,Price,Apartment Type,Metro Station,Minutes To Metro,Region,Number Of Rooms,Area,Living Area,Kitchen Area,Floor,Number Of Floors,Renovation
0,0,6300000.0,Вторичка,Опалиха,6.0,Московская Область,1.0,30.6,11.1,8.5,25.0,25,Косметический
1,1,9000000.0,Вторичка,Павшино,2.0,Московская Область,1.0,49.2,20.0,10.0,6.0,15,В Европейском Стиле
2,2,11090000.0,Вторичка,Мякинино,14.0,Московская Область,1.0,44.7,16.2,13.1,10.0,25,Косметический
3,3,8300000.0,Вторичка,Строгино,8.0,Московская Область,1.0,35.1,16.0,11.0,12.0,33,В Европейском Стиле
4,4,6450000.0,Вторичка,Опалиха,6.0,Московская Область,1.0,37.7,15.2,4.0,5.0,5,Без Ремонта


<H2>Обработка Данных Таблицы. Кодирование Категориальных Признаков (String -> Int).</H2>

In [20]:
Features = ["Apartment Type", "Metro Station", "Minutes To Metro", "Region", "Number Of Rooms", 
            "Area", "Living Area", "Kitchen Area", "Floor", "Number Of Floors", "Renovation"]
FeaturesFloat = ["Minutes To Metro", "Number Of Rooms", "Area", "Living Area", "Kitchen Area", "Floor"]
Data["Price"] = Data["Price"].astype(int)
Count = 0
for Num in range(len(FeaturesFloat)):
    Data[FeaturesFloat[Count]] = Data[FeaturesFloat[Count]].astype(int)
    Count = Count + 1
FeaturesString = ["Apartment Type", "Metro Station", "Region", "Renovation"]
Count = 0
for Num in range(len(FeaturesString)):
    Encoder = sklearn.preprocessing.LabelEncoder()
    Data[FeaturesString[Count] + " Label"] = Data[FeaturesString[Count]]
    Data[FeaturesString[Count]] = Encoder.fit_transform(Data[FeaturesString[Count]])
    Count = Count + 1
display(Data.head())

,Unnamed: 0,Price,Apartment Type,Metro Station,Minutes To Metro,Region,Number Of Rooms,Area,Living Area,Kitchen Area,Floor,Number Of Floors,Renovation,Apartment Type Label,Metro Station Label,Region Label,Renovation Label
0,0,6300000,0,176,6,1,1,30,11,8,25,25,3,Вторичка,Опалиха,Московская Область,Косметический
1,1,9000000,0,183,2,1,1,49,20,10,6,15,1,Вторичка,Павшино,Московская Область,В Европейском Стиле
2,2,11090000,0,148,14,1,1,44,16,13,10,25,3,Вторичка,Мякинино,Московская Область,Косметический
3,3,8300000,0,251,8,1,1,35,16,11,12,33,1,Вторичка,Строгино,Московская Область,В Европейском Стиле
4,4,6450000,0,176,6,1,1,37,15,4,5,5,0,Вторичка,Опалиха,Московская Область,Без Ремонта


In [21]:
Count = 0
LabelValues = []
for Num in range(len(FeaturesString)):
    LabelValues.append(list(Data[FeaturesString[Count] + " Label"].unique()))
    Count = Count + 1
EncodedValues = []
Count = 0
for Num in range(len(FeaturesString)):
    EncodedValues.append(list(Data[FeaturesString[Count]].unique()))
    Count = Count + 1

<H1>Создание Моделей Регрессии. Предсказание Стоимости Недвижимости.</H1>

<H2>Разделение Данных На Обучающую И Тестовую Выборку.</H2>

In [22]:
X = Data[Features]
Y = Data["Price"]
X_Train, X_Test, Y_Train, Y_Test = sklearn.model_selection.train_test_split(X, Y)

<H2>Обучение Модели Регрессии RandomForestClassifier Из Библиотеки Sklearn.Ensemble. Точность Модели.</H2>

In [23]:
RF_Model = sklearn.ensemble.RandomForestRegressor(1000)
RF_Model.fit(X_Train, Y_Train)
RF_Y_Pred = RF_Model.predict(X_Test)
RF_R2 = sklearn.metrics.r2_score(Y_Test, RF_Y_Pred)
RF_MAE = sklearn.metrics.mean_absolute_error(Y_Test, RF_Y_Pred)
RF_MAPE = sklearn.metrics.mean_absolute_percentage_error(Y_Test, RF_Y_Pred)
print("RandomForestRegressor R^2:", RF_R2)
print("RandomForestRegressor MAE:", RF_MAE)
print("RandomForestRegressor MAPE:", RF_MAPE)

RandomForestRegressor R^2: 0.781152617610225
RandomForestRegressor MAE: 8833715.985963615
RandomForestRegressor MAPE: 0.15707421730166446


<H2>Создание Модели Регрессии Sequential Из Библиотеки Tensorflow Keras.</H2>

In [24]:
Seq_Model = keras.Sequential([
    keras.layers.Input([11]),
    keras.layers.Dense(512),
    keras.layers.ReLU(),
    keras.layers.Dense(256),
    keras.layers.ReLU(),
    keras.layers.Dense(128),
    keras.layers.ReLU(),
    keras.layers.Dense(64),
    keras.layers.ReLU(),
    keras.layers.Dense(32),
    keras.layers.ReLU(),
    keras.layers.Dense(1)
])

<H2>Обучение Модели. Точность Модели.</H2>

In [25]:
Seq_Model.compile(keras.optimizers.Adam(), keras.losses.MAE, None, [keras.losses.MAE])
Seq_Model.fit(X_Train, Y_Train, None, 200, 0)
Seq_Y_Pred = Seq_Model.predict(X_Test, None, 0)
Seq_R2 = sklearn.metrics.r2_score(Y_Test, Seq_Y_Pred)
Seq_MAE = sklearn.metrics.mean_absolute_error(Y_Test, Seq_Y_Pred)
Seq_MAPE = sklearn.metrics.mean_absolute_percentage_error(Y_Test, Seq_Y_Pred)
print("Sequential R^2:", Seq_R2)
print("Sequential MAE:", Seq_MAE)
print("Sequential MAPE:", Seq_MAPE)

Sequential R^2: 0.7288596630096436
Sequential MAE: 11739149.409992944
Sequential MAPE: 0.20121744976690278


<H2>Выгрузка Моделей В Файлы.</H2>

In [ ]:
joblib.dump(RF_Model, 'RFModel.pkl')
Seq_Model.save('KerasSequential.keras')

<H2>Загрузка Моделей Из Файлов.</H2>

In [ ]:
RF_Model = joblib.load('RFModel.pkl')
Seq_Model = keras.models.load_model('KerasSequential.keras')

<H2>Ввод Пользовательских Данных.</H2>

In [ ]:
Count = 0
print("Тип. Доступные Значения:", LabelValues[0])
ApartmentTypeLabel = input("Введите Тип:")
ApartmentType = EncodedValues[Count][LabelValues[Count].index(ApartmentTypeLabel)]
Count = Count + 1
print("Станция Метро. Доступные Значения:", LabelValues[1])
MetroStationLabel = input("Введите Название:")
MetroStation = EncodedValues[Count][LabelValues[Count].index(MetroStationLabel)]
Count = Count + 1
print("Минут До Метро.")
MinutesToMetro = int(input("Введите Число:"))
print("Регион. Доступные Значения:", LabelValues[2])
RegionLabel = input("Введите Регион:")
Region = EncodedValues[Count][LabelValues[Count].index(RegionLabel)]
Count = Count + 1
print("Количество Комнат.")
NumberOfRooms = int(input("Введите Число:"))
print("Квадратные Метры.")
Area = int(input("Введите Число:"))
print("Жилая Площадь.")
LivingArea = int(input("Введите Число:"))
print("Площадь Кухни.")
KitchenArea = int(input("Введите Число:"))
print("Этаж.")
Floor = int(input("Введите Число:"))
print("Количество Этажей.")
NumberOfFloors = int(input("Введите Число:"))
print("Тип Ремонта. Доступные Значения:", LabelValues[3])
RenovationLabel = input("Введите Тип Ремонта:")
Renovation = EncodedValues[Count][LabelValues[Count].index(RenovationLabel)]
DataToPredict = pd.DataFrame({"Apartment Type": [ApartmentType], "Metro Station": [MetroStation], "Minutes To Metro": [MinutesToMetro], "Region": [Region],
                              "Number Of Rooms": [NumberOfRooms], "Area": [Area], "Living Area": [LivingArea], "Kitchen Area": [KitchenArea], "Floor": [Floor], 
                              "Number Of Floors": [NumberOfFloors], "Renovation": [Renovation]})

<H2>Предсказание Стоимости Недвижимости На Основании Введенных Данных Моделями RandomForestRegressor, Sequential.</H2>

In [ ]:
PredictedPriceRF = RF_Model.predict(DataToPredict)
print("Цена RandomForest:", PredictedPriceRF)
PredictedPriceSeq = Seq_Model.predict(DataToPredict, None, 0)
print("Цена Sequential:", PredictedPriceSeq)

<H1>Создание Моделей Классификации. Предсказание Типа Недвижимости (Новостройка, Вторичка).</H1>

<H2>Разделение Данных На Обучающую И Тестовую Выборки.</H2>

In [ ]:
X_Classifier = Data[["Price", "Metro Station", "Minutes To Metro", "Region", "Number Of Rooms", 
                     "Area", "Living Area", "Kitchen Area", "Floor", "Number Of Floors", "Renovation"]]
Y_Classifier = Data["Apartment Type"]
X_Classifier_Train, X_Classifier_Test, Y_Classifier_Train, Y_Classifier_Test = sklearn.model_selection.train_test_split(X_Classifier, Y_Classifier)

<H2>Обучение Модели Классификации RandomForestClassifier Из Библиотеки Sklearn.Ensemble. Точность Модели.</H2>

In [ ]:
RF_Classifier = sklearn.ensemble.RandomForestClassifier(1000)
RF_Classifier.fit(X_Classifier_Train, Y_Classifier_Train)
Y_RF_Classifier_Pred = RF_Classifier.predict(X_Classifier_Test)
RF_Classifier_Accuracy = sklearn.metrics.accuracy_score(Y_Classifier_Test, Y_RF_Classifier_Pred)
print("RF Classifier Accuracy:", RF_Classifier_Accuracy)

<H2>Создание Модели Классификации Sequential Из Библиотеки Tensorflow Keras. Активационная Функция Sigmoid.</H2>

In [ ]:
Seq_Classifier = keras.Sequential([
    keras.layers.Input([11]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(512),
    keras.layers.ReLU(),
    keras.layers.Dense(256),
    keras.layers.ReLU(),
    keras.layers.Dense(128),
    keras.layers.ReLU(),
    keras.layers.Dense(64),
    keras.layers.ReLU(),
    keras.layers.Dense(32),
    keras.layers.ReLU(),
    keras.layers.Dense(1, activation="sigmoid")
])

<H2>Обучение Модели.</H2>

In [ ]:
Seq_Classifier.compile(keras.optimizers.Adam(), keras.losses.binary_crossentropy, None, [keras.metrics.Accuracy])
Seq_Classifier.fit(X_Classifier_Train, Y_Classifier_Train, None, 50, 0)

<H2>Точность Модели.</H2>

In [ ]:
Y_Seq_Classifier_Pred = Seq_Classifier.predict(X_Classifier_Test, None, 0)
Y_Seq_Classifier_Pred = pd.DataFrame(Y_Seq_Classifier_Pred)
Y_Seq_Classifier_Pred[(Y_Seq_Classifier_Pred.iloc[:, 0] < 0.5)] = 0
Y_Seq_Classifier_Pred[(Y_Seq_Classifier_Pred.iloc[:, 0] >= 0.5)] = 1
Seq_Classifier_Accuracy = sklearn.metrics.accuracy_score(Y_Classifier_Test, Y_Seq_Classifier_Pred)
print("Sequential Classifier Accuracy:", Seq_Classifier_Accuracy)

<H2>Кодирование Целевого Столбца Apartment Type Методом OneHot. Разделение Данных На Обучающую И Тестовую Выборки.</H2>

In [ ]:
HotEncoder = sklearn.preprocessing.OneHotEncoder(sparse_output=False)
X_Classifier_Hot = Data[["Price", "Metro Station", "Minutes To Metro", "Region", "Number Of Rooms", 
                     "Area", "Living Area", "Kitchen Area", "Floor", "Number Of Floors", "Renovation"]]
Y_Classifier_Hot = HotEncoder.fit_transform(Data[["Apartment Type"]])
Y_Classifier_Hot = pd.DataFrame(Y_Classifier, columns = HotEncoder.get_feature_names_out(["Apartment Type"]))
X_Classifier_Train_Hot, X_Classifier_Test_Hot, Y_Classifier_Train_Hot, Y_Classifier_Test_Hot = sklearn.model_selection.train_test_split(X_Classifier_Hot, Y_Classifier_Hot)

<H2>Создание Модели Классификации Sequential Из Библиотеки Tensorflow Keras. Активационная Функция Softmax.</H2>

In [ ]:
Seq_MultiClassifier = keras.Sequential([
    keras.layers.Input([11]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(512),
    keras.layers.ReLU(),
    keras.layers.Dense(256),
    keras.layers.ReLU(),
    keras.layers.Dense(128),
    keras.layers.ReLU(),
    keras.layers.Dense(64),
    keras.layers.ReLU(),
    keras.layers.Dense(32),
    keras.layers.ReLU(),
    keras.layers.Dense(2, activation="softmax")
])

<H2>Обучение Модели.</H2>

In [ ]:
Seq_MultiClassifier.compile(keras.optimizers.Adam(), keras.losses.categorical_crossentropy, None, [keras.metrics.Accuracy])
Seq_MultiClassifier.fit(X_Classifier_Train, Y_Classifier_Train, None, 50, 1)

In [ ]:
def CompareColumns(Row):
    Row = pd.Series(Row, index=Row.index)
    MaxValue = Row.max()
    Row[(Row != MaxValue)] = 0
    Row[(Row == MaxValue)] = 1
    return Row

<H2>Точность Модели.</H2>

In [ ]:
Y_Seq_MultiClassifier_Pred = Seq_MultiClassifier.predict(X_Classifier_Test, None, 0)
Y_Seq_MultiClassifier_Pred = pd.DataFrame(Y_Seq_MultiClassifier_Pred)
Y_Seq_MultiClassifier_Pred.columns = Y_Classifier_Hot.columns
Y_Seq_MultiClassifier_Pred = Y_Seq_MultiClassifier_Pred.apply(lambda Row: CompareColumns(Row), axis=1)
Y_Seq_MultiClassifier_Pred = pd.DataFrame(Y_Seq_MultiClassifier_Pred)
Y_Seq_MultiClassifier_Accuracy = sklearn.metrics.accuracy_score(Y_Classifier_Test_Hot, Seq_Y_MultiClassifier_Pred)
print("Sequential MultiClassifier Accuracy:", Y_Seq_MultiClassifier_Accuracy)